In [1]:
import pandas as pd
import numpy as np
import json
import yaml
from statsmodels.stats.contingency_tables import mcnemar
from statsmodels.stats.contingency_tables import cochrans_q
from scipy.stats import combine_pvalues
from scipy.stats import norm

In [2]:
def read_yaml_answers(file_path):
    answers = []
    with open(file_path, "r") as f:
        documents = list(yaml.safe_load_all(f))  # handles both single list or multiple docs
        for doc in documents:
            if isinstance(doc, list):  # case: YAML file is a list of dicts
                for item in doc:
                    if isinstance(item, dict) and "answer" in item:
                        answers.append(item["answer"])
            elif isinstance(doc, dict):  # case: multiple `---` documents
                if "answer" in doc:
                    answers.append(doc["answer"])
    return answers

def read_unstructured_parsed(file_path):
    with open(file_path, "r") as f:
        data = json.load(f)

    answers = []
    if isinstance(data, list):
        for obj in data:
            if isinstance(obj, dict) and "parsed" in obj:
                answers.append(obj["parsed"])
    return answers

In [3]:
# Structured answers
yaml_standard_answers = read_yaml_answers("gcf_results/test_yaml_0.yaml")
yaml_artist_answers = read_yaml_answers("gcf_results/test_yaml_artist_0.yaml")
yaml_chef_answers = read_yaml_answers("gcf_results/test_yaml_chef_0.yaml")
yaml_detective_answers = read_yaml_answers("gcf_results/test_yaml_detective_0.yaml")
yaml_judge_answers = read_yaml_answers("gcf_results/test_yaml_judge_0.yaml")
# Unstructured answers
unstructured_standard_answers = read_unstructured_parsed("gcf_results/test_unstructured_parsed_0.json")
unstructured_artist_answers = read_unstructured_parsed("gcf_results/test_unstructured_artist_parsed_0.json")
unstructured_chef_answers = read_unstructured_parsed("gcf_results/test_unstructured_chef_parsed_0.json")
unstructured_detective_answers = read_unstructured_parsed("gcf_results/test_unstructured_detective_parsed_0.json")
unstructured_judge_answers = read_unstructured_parsed("gcf_results/test_unstructured_judge_parsed_0.json")

In [4]:
label = pd.read_csv("data/fac_data_balanced.csv")

unstructured_standard_results = []
unstructured_artist_results = []
unstructured_chef_results = []
unstructured_detective_results = []
unstructured_judge_results = []

yaml_standard_results = []
yaml_artist_results = []
yaml_chef_results = []
yaml_detective_results = []
yaml_judge_results = []

for i in range(len(unstructured_standard_answers)):
    label_tmp = label['label'][i]
    if label_tmp == 'Flase':
        label_tmp = 'no'
    if label_tmp == 'True':
        label_tmp = 'yes' 
    # Unstructured answers
    if label_tmp.lower() == unstructured_standard_answers[i].lower():
        unstructured_standard_results.append(1)
    else:
        unstructured_standard_results.append(0)
        
    if label_tmp.lower() == unstructured_artist_answers[i].lower():
        unstructured_artist_results.append(1)
    else:
        unstructured_artist_results.append(0)
        
    if label_tmp.lower() == unstructured_chef_answers[i].lower():
        unstructured_chef_results.append(1)
    else:
        unstructured_chef_results.append(0)
        
    if label_tmp.lower() == unstructured_detective_answers[i].lower():
        unstructured_detective_results.append(1)
    else:
        unstructured_detective_results.append(0)
        
    if label_tmp.lower() == unstructured_judge_answers[i].lower():
        unstructured_judge_results.append(1)
    else:
        unstructured_judge_results.append(0)
    
    # Structured answers
    if str(yaml_standard_answers[i]) == 'True':
        yaml_standard_answers[i] = 'yes'
    if str(yaml_standard_answers[i]) == 'False':
        yaml_standard_answers[i] = 'no'    
    if label_tmp.lower() == yaml_standard_answers[i].lower():
        yaml_standard_results.append(1)
    else:
        yaml_standard_results.append(0)   

    if str(yaml_artist_answers[i]) == 'True':
        yaml_artist_answers[i] = 'yes'
    if str(yaml_artist_answers[i]) == 'False':
        yaml_artist_answers[i] = 'no'  
    if label_tmp.lower() == yaml_artist_answers[i].lower():
        yaml_artist_results.append(1)
    else:
        yaml_artist_results.append(0) 

    if str(yaml_chef_answers[i]) == 'True':
        yaml_chef_answers[i] = 'yes'
    if str(yaml_chef_answers[i]) == 'False':
        yaml_chef_answers[i] = 'no'  
    if label_tmp.lower() == yaml_chef_answers[i].lower():
        yaml_chef_results.append(1)
    else:
        yaml_chef_results.append(0) 
        
    if str(yaml_detective_answers[i]) == 'True':
        yaml_detective_answers[i] = 'yes'
    if str(yaml_detective_answers[i]) == 'False':
        yaml_detective_answers[i] = 'no'          
    if label_tmp.lower() == yaml_detective_answers[i].lower():
        yaml_detective_results.append(1)
    else:
        yaml_detective_results.append(0) 
        
    if str(yaml_judge_answers[i]) == 'True':
        yaml_judge_answers[i] = 'yes'
    if str(yaml_judge_answers[i]) == 'False':
        yaml_judge_answers[i] = 'no'          
    if label_tmp.lower() == yaml_judge_answers[i].lower():
        yaml_judge_results.append(1)
    else:
        yaml_judge_results.append(0) 

In [5]:
# Test whether output format significantly influences LLMs' generation
print(f"Averaged accuracy of unstructured format with diverse instruction: {format((sum(unstructured_standard_results)+sum(unstructured_artist_results)+sum(unstructured_chef_results)+sum(unstructured_detective_results)+sum(unstructured_judge_results))/(5*len(unstructured_standard_results)),'.3f')}")
print(f"Averaged accuracy of YAML format with diverse instruction: {format((sum(yaml_standard_results)+sum(yaml_artist_results)+sum(yaml_chef_results)+sum(yaml_detective_results)+sum(yaml_judge_results))/(5*len(yaml_standard_results)),'.3f')}")
data_subsets = [
    # Stratum of control on standard instruction
    (np.array(yaml_standard_results), np.array(unstructured_standard_results)),
    # Stratum of control on artist instruction
    (np.array(yaml_artist_results), np.array(unstructured_artist_results)),
    # Stratum of control on chef instruction
    (np.array(yaml_chef_results), np.array(unstructured_chef_results)),
    # Stratum of control on detective instruction
    (np.array(yaml_detective_results), np.array(unstructured_detective_results)),
    # Stratum of control on judge instruction
    (np.array(yaml_judge_results), np.array(unstructured_judge_results)),
]

p_values = []

for i, (correct_D, correct_E) in enumerate(data_subsets, 1):
    # Build contingency table for this stratum
    table = np.zeros((2, 2), dtype=int)
    for d, e in zip(correct_D, correct_E):
        table[d, e] += 1

    # Run McNemar's test
    result = mcnemar(table, exact=True)
    p_values.append(result.pvalue)

    # print(f"Stratum {i} contingency table:\n{table}")
    # print(f"Stratum {i} McNemar p-value: {result.pvalue:.5f}\n")

# Combine p-values using Stouffer’s method with correlation adjustment
# Convert p-values to Z-scores (two-sided)
z_scores = norm.isf(np.array(p_values) / 2)  # inverse survival function

# Assume weights (e.g., uniform or by number of samples per stratum)
weights = np.ones_like(z_scores)
# Assume correlation between strata (due to shared samples)
rho = 0.3
k = len(p_values)
R = np.full((k, k), rho)
np.fill_diagonal(R, 1)  # correlation matrix

# Stouffer's Z with correlation adjustment
numerator = np.sum(weights * z_scores)
denominator = np.sqrt(np.dot(weights, R @ weights))
z_combined = numerator / denominator
p_combined = 2 * norm.sf(abs(z_combined))

# print(f"Combined Stouffer Z: {z_combined:.3f}")
print(f"Combined p-value across strata (Stouffer’s method): {p_combined:.3f}")

Averaged accuracy of unstructured format with diverse instruction: 0.693
Averaged accuracy of YAML format with diverse instruction: 0.713
Combined p-value across strata (Stouffer’s method): 0.287


In [6]:
# Test whether instruction significantly influences LLMs' generation
print(f"Accuracy of unstructured & YAML format with standard instruction: {format((sum(unstructured_standard_results)+sum(yaml_standard_results))/(2*len(yaml_standard_results)),'.3f')}")
print(f"Averaged accuracy of unstructured & YAML format with intervened instruction: {format((sum(unstructured_artist_results)+sum(unstructured_chef_results)+sum(unstructured_detective_results)+sum(unstructured_judge_results)+sum(yaml_artist_results)+sum(yaml_chef_results)+sum(yaml_detective_results)+sum(yaml_judge_results))/(8*len(yaml_standard_results)),'.3f')}")

stratum_yaml = np.array([yaml_standard_results, yaml_artist_results, yaml_chef_results, yaml_detective_results, yaml_judge_results]).transpose()
stratum_unstructured = np.array([unstructured_standard_results, unstructured_artist_results, unstructured_chef_results, unstructured_detective_results, unstructured_judge_results]).transpose()
data_strata = [stratum_yaml, stratum_unstructured]

# Run Cochran's Q test on each stratum
p_values = []
# print("Cochran's Q test per stratum:")
for i, data in enumerate(data_strata):
    result = cochrans_q(data)
    p_values.append(result.pvalue)
    # print(f"  Stratum {i+1}: Q = {result.statistic:.4f}, p = {result.pvalue:.5f}")

# Combine p-values using Stouffer’s method with correlation adjustment
# Convert p-values to Z-scores (two-sided)
z_scores = norm.isf(np.array(p_values) / 2)  # inverse survival function

# Assume weights (e.g., uniform or by number of samples per stratum)
weights = np.ones_like(z_scores)
# Assume correlation between strata (due to shared samples)
rho = 0.3
k = len(p_values)
R = np.full((k, k), rho)
np.fill_diagonal(R, 1)  # correlation matrix

# Stouffer's Z with correlation adjustment
numerator = np.sum(weights * z_scores)
denominator = np.sqrt(np.dot(weights, R @ weights))
z_combined = numerator / denominator
p_combined = 2 * norm.sf(abs(z_combined))

# print(f"Combined Stouffer Z: {z_combined:.3f}")
print(f"Combined p-value across strata (Stouffer’s method): {p_combined:.3f}")

Accuracy of unstructured & YAML format with standard instruction: 0.701
Averaged accuracy of unstructured & YAML format with intervened instruction: 0.703
Combined p-value across strata (Stouffer’s method): 0.287
